In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE57376"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE57376"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE57376.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57376.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE57376.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Synovial biopsies from RA and PsA patients and skin biopsies from Psoriasis patients under Infliximab treatment"
!Series_summary	"Object: to understand Infliximab treatment effect on the molecular expression of tissue at disease site"
!Series_overall_design	"4mm punch biopsies were performed on involved and uninvolved skin at baseline in 5 Ps patients. A repeat biopsy was performed at week 2 after IFX therapy at a site adjacent to the baseline biopsy of involved skin. Synovial biopsies were performed on the knee of 3 RA and 3 PsA paired-subjects with a Parker Pearson biopsy needle (Dyna Medical, London, Canada) under ultrasound guidance at baseline and repeated on the same knee at week 10"
Sample Characteristics Dictionary:
{0: ['Sex: F', 'Sex: M'], 1: ['age: 51', 'age: 28', 'age: 46', 'age: 57', 'age: 61', 'age: 35', 'age: 19', 'age: 67', 'age: 38', 'age: 55', 'age: 39', 'age: 44', 'age: 52'], 2: ['sample type: biopsy'], 3: ['tissue: knee', 'tiss

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Check if gene expression data is likely available
is_gene_available = True  # Yes, based on the background information which mentions "molecular expression"

# Identify row indices for trait, age, and gender
trait_row = 5  # The disease status is in row 5
age_row = 1    # Age is in row 1
gender_row = 0  # Gender (Sex) is in row 0

# Define conversion functions
def convert_trait(value):
    """Convert trait value to binary (1 for Psoriatic Arthritis, 0 for others)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == "psoriatic arthritis":
        return 1
    elif value.lower() in ["rheumatoid arthritis", "psoriasis"]:
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() in ['f', 'female']:
        return 0
    elif value.lower() in ['m', 'male']:
        return 1
    return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata using validate_and_save_cohort_info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Assuming clinical_data is already defined from previous steps
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait, 
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM1381406': [0.0, 51.0, 0.0], 'GSM1381407': [0.0, 28.0, 0.0], 'GSM1381408': [0.0, 46.0, 1.0], 'GSM1381409': [0.0, 57.0, 1.0], 'GSM1381410': [0.0, 61.0, 1.0], 'GSM1381411': [0.0, 35.0, 0.0], 'GSM1381412': [0.0, 28.0, 0.0], 'GSM1381413': [0.0, 19.0, 0.0], 'GSM1381414': [0.0, 28.0, 0.0], 'GSM1381415': [0.0, 61.0, 1.0], 'GSM1381416': [0.0, 57.0, 1.0], 'GSM1381417': [0.0, 35.0, 0.0], 'GSM1381418': [0.0, 19.0, 0.0], 'GSM1381419': [1.0, 67.0, 1.0], 'GSM1381420': [1.0, 38.0, 0.0], 'GSM1381422': [1.0, 55.0, 1.0], 'GSM1381423': [1.0, 39.0, 1.0], 'GSM1381424': [1.0, 55.0, 1.0], 'GSM1381425': [0.0, 19.0, 0.0], 'GSM1381426': [0.0, 61.0, 1.0], 'GSM1381427': [0.0, 28.0, 0.0], 'GSM1381428': [0.0, 35.0, 0.0], 'GSM1381429': [0.0, 57.0, 1.0], 'GSM1381430': [0.0, 51.0, 0.0], 'GSM1381431': [0.0, 28.0, 0.0], 'GSM1381432': [0.0, 28.0, 0.0], 'GSM1381433': [0.0, 46.0, 1.0], 'GSM1381434': [1.0, 44.0, 1.0], 'GSM1381435': [1.0, 67.0, 1.0], 'GSM1381436': [1.0, 52.0, 0.0], 'GSM13814

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (indicated by the "_PM_" pattern)
# They are not standard human gene symbols and will need to be mapped to gene symbols
# The "_PM_" format is typical of Affymetrix microarray platforms

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# Identify which columns contain probe IDs and gene symbols
# From the preview, we can see:
# 'ID' column contains the probe identifiers (same format as gene_data.index)
# 'Gene Symbol' column contains the actual gene symbols

# Get the mapping dataframe with probe ID and gene symbol
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# Preview the mapping data
print("Mapping data preview:")
print(preview_df(mapping_data))

# Apply the gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Preview the resulting gene expression data
print("\nGene expression data preview (first 5 genes, 5 samples):")
print(gene_data.iloc[:5, :5])

# Print the number of genes and samples in the final dataset
print(f"\nFinal gene expression dataset dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")


Mapping data preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'Gene': ['DDR1', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Gene expression data preview (first 5 genes, 5 samples):
       GSM1381406  GSM1381407  GSM1381408  GSM1381409  GSM1381410
Gene                                                             
A1BG       4.1707      4.3218      3.8129      4.1707      4.1545
A1CF       3.8796      4.2474      4.3014      3.9077      3.9654
A2BP1     12.4500     12.3720     12.9834     13.5768     13.4549
A2LD1      8.6088      9.0508      8.7998      8.5940      8.4884
A2M       17.7359     17.8611     18.0590     16.3133     15.3769

Final gene expression dataset dimensions: 18989 genes × 32 samples


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")

Gene data saved to ../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57376.csv
Linked data shape before handling missing values: (32, 18625)


Linked data shape after handling missing values: (32, 18625)
For the feature 'Psoriatic_Arthritis', the least common label is '1.0' with 10 occurrences. This represents 31.25% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.0
  50% (Median): 45.0
  75%: 55.5
Min: 19.0
Max: 67.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 16 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Psoriatic_Arthritis/cohort_info.json
Data is usable. Saving to ../../output/preprocess/Psoriatic_Arthritis/GSE57376.csv
